# Solution to tutorial exercises

## 1_Basics.ipynb

### Exercise 1

In [ ]:
src, dst = zip(*edge_list)
G.add_edges(src, dst)

### Exercise 2

In [ ]:
G.edata['broness'] += torch.randn((G.number_of_edges(),)) * 0.1

## 2_MessagePassing.ipynb

### Exercise 1

In [ ]:
num_invited = int(G.ndata['invited'].sum())
print("{} members invited".format(num_invited))

while num_invited < 34:
    G.send(G.edges(), message_func)
    G.recv(G.nodes(), reduce_func)
    num_invited = int(G.ndata['invited'].sum())
    print("{} members invited".format(num_invited))

## 3_GCN.ipynb
### Exercise 1

In [ ]:
def gcn_message(edges):
    # Send features of source nodes as message.
    # Hint 1: you can access the feature dictionary of source node, destination node and edges with
    #         edges.src, edges.dst, edges.data
    # Hint 2: name of the node feature is decided when you set the feature in GCN forward function
    return {'msg': edges.src['h']}

def gcn_reduce(nodes):
    # Aggregate messages from neighbors by summation.
    # Hint 1: messages are delivered into mailbox and you can access it using nodes.mailbox
    # Hint 2: mailbox is the same as the feature dictionary you return in gcn_message
    return {'h': nodes.mailbox['msg'].sum(dim=1)}

### Exercise 2

In [ ]:
# You need to make two changes:

# 1. Get the degrees of nodes and make them a node feature. You can do this in forward function of GCN, 
#    or during pre-processing
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
    
    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first perform linear transformation
        h = self.linear(inputs)
        # set the node features
        g.ndata['h'] = h
        # set node degrees
        g.ndata['deg'] = g.in_degrees(g.nodes())
        # trigger message passing, gcn_message and gcn_reduce will be defined later
        g.send(g.edges(), gcn_message)
        g.recv(g.nodes(), gcn_reduce)
        # get the result node features
        h = g.ndata.pop('h')
        return h
    
# 2. Update the message function and use node degree to compute normalization factor
def gcn_message(edges):
    return {'msg': torch.sqrt(edges.src['deg'] * edges.dst['deg']) * edges.src['h']}